### Step 1 : Set image folder.

In [17]:
# 👇 Use your own image folder path.
image_folder_path = '../images'

### Step 2 : Pre-process images.

In [18]:
import os
import base64
from collections import namedtuple

ImageDate = namedtuple('ImageDate', ['filename_without_extension', 'extension', 'base64_string', 'caption'])


def convert_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

def get_file_type(filename):
    return os.path.splitext(filename)[1][1:]  # get file extension without dot

def get_filename_without_extension(filename):
    return os.path.splitext(filename)[0]  # get filename without extension

def convert_images_in_folder(folder_path):
    encoded_images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            file_path = os.path.join(folder_path, filename)
            base64_string = convert_image_to_base64(file_path)
            file_type = get_file_type(filename)
            filename_without_extension = get_filename_without_extension(filename)
            encoded_images.append(ImageDate(filename_without_extension, file_type, base64_string, ""))
    return encoded_images

base64_images = convert_images_in_folder(image_folder_path)

# 🐞 uncomment to debug
# for item in base64_images:
#     print(item[0], item[1], item[2], item[3])

print(f'{len(base64_images)} image(s) processed.')


2 image(s) processed.


### Step 3 : Call Azure OpenAI services.

❗️❗️ Don't foget to config your Azure OpenAI key:
- Add `AZURE_OPENAI_API_KEY={your_azure_openai_key_here}` to `.env` file;
- If it doesn't exist, create one in this project root folder.

In [16]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version="2023-07-01-preview",
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint="https://openai-ifbox-ai.openai.azure.com",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

for item in base64_images:
  completion = client.chat.completions.create(
      model="gpt-4-vision-preview",  # Your deployment name.
      messages=[
      {
        "role": "system",
        
        "content": 
        # Edit this system message if needed.
        """
        You are going to help people to describe the image.
        You should always in english start with "the image is showing"      
        """
      },
      {
        "role": "user",
        "content": [
          # {
          #   "type": "text", 
          #   "text": "What’s in this image?"
          # },
          {
            "type": "image_url",

            # Using the image url.
            # "image_url": { "url": "https://ifbox.ai/static/brand.e18dbb5f.png" },

            # Using the base64 encoded image from the file.
            "image_url": { "url": f"data:image/{(item as ImageDate)};base64,{item[3]}" },
          },
        ],
      }
    ],
    max_tokens=500,
  )
  print(completion.choices[0].message.content)

The image is showing the word "GIFBOX" with a stylized icon resembling a square or box with a blue to purple gradient outline to the left of the text. The text itself is in a bold, sans-serif typeface, with the letters "IFBOX" in black and the letter "G" in a hollow, outlined style matching the gradient of the icon.
The image is showing a large futuristic spacecraft positioned at the forefront, with a group of three individuals facing towards it, possibly pilots or engineers. The scene appears to be set in a vast hangar or docking station with intricate structures extending into the background where the overcast sky looms above. The lighting suggests a setting sun or artificial illumination casting a warm glow on the scene, highlighting the orange and yellow tones of the spacecraft and the surrounding environment. The presence of a smaller vehicle on the right side of the image gives a sense of scale to the massive spacecraft. It is a visually striking image that conveys a sense of ant

### Step 4 : Post-process.

### Step 5 : Humun review.